# **Rescaling and Dimension Adjustment of 3D Images**

---

<font size = 4>This notebook facilitates the adjustment of x- and y-axis dimensions in 3D images to the desired ones, calculates the new pixel spacing, and applies linear pixel interpolation to generate the rescaled images.

## **1. Dependencies**
---


### **1.1. Install dependencies**
---
<font size = 4>

In [1]:
#@markdown ##Install dependencies
! pip install -q readlif
! pip install -q SimpleITK
! pip install -q aicsimageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.4/248.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.2/405.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.1/195.1 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 202.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 

### **1.2. Load key dependencies**
---
<font size = 4>

In [3]:
#@markdown ##Load dependencies

from readlif.reader import LifFile
import numpy as np
import tifffile as tf
import os
import SimpleITK as sitk
from aicsimageio import AICSImage
import shutil
from skimage import io, img_as_uint
from skimage.util import random_noise
import xml.etree.ElementTree as ET
import ipywidgets as widgets
from IPython.display import display

## **2. Initialise the Colab session**
---

### **2.1. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [4]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Follow the instructions.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**<font size = 4> If you cannot see your files, reactivate your session by connecting to your hosted runtime.**


<img width="40%" alt ="Example of image detection with retinanet." src="https://github.com/HenriquesLab/ZeroCostDL4Mic/raw/master/Wiki_files/connect_to_hosted.png"><figcaption> Connect to a hosted runtime. </figcaption>

## **3. Select your paths**

---


<font size = 5> **Paths for folder of input images**

<font size = 4>**`base_path`:** this is the path where the folder with your  images is located in your google drive and where the output will be saved.

<font size = 4>**`images_folder`:** this is the folder path of your images.




 To find the path of the folder containing the respective images, go to your Files on the left of the notebook, navigate to the folder containing your files and copy the path by right-clicking on the folder, **Copy path** and pasting it into the right box below.

In [5]:
#@markdown ###Path of the original images:

# base folder where the GT image is
base_path = "/content/gdrive/MyDrive/Colab Notebooks/res" #@param {type:"string"}

# image file name
images_folder = "/content/gdrive/MyDrive/Colab Notebooks/res/train_up" #@param {type:"string"}

output_folder_name = widgets.Text(value='', description='Enter output folder name:')
output_folder_name.style.description_width = '150px'
output_folder_name.layout.width = '600px'

def check_output_folder(base_path, folder_name):
    return folder_name in os.listdir(base_path)

def on_button_clicked(b):
    new_folder_name = output_folder_name.value.strip()
    if new_folder_name:
        if check_output_folder(base_path, new_folder_name):
            print('This folder already exists. Please choose another name.')
            output_folder_name.value = ''
        else:
            print('Folder name is available:', new_folder_name)
    else:
        print('Please provide a name for the output folder.')

button = widgets.Button(description='Check Folder Name')
button.on_click(on_button_clicked)

display(output_folder_name, button)

Text(value='', description='Enter output folder name:', layout=Layout(width='600px'), style=DescriptionStyle(d…

Button(description='Check Folder Name', style=ButtonStyle())

Folder name is available: rescaled_imgs


# **4. Image rescaling**
---

<font size = 4>**`rescaling_factor`:** this numerical value acts as a multiplier for the original x and y dimensions of the images, allowing for upscaling or downscaling. For instance, applying a factor of 10 will increase the size, while a facotr of 0.1 will decrease it. It is important to note that after multiplication, only the non-decimal portion determines the new dimensions (no rounding occurs). For example, if the initial x and y dimensions are both 2048, and the rescaling factor is set to 0.1, the resulting dimensions will be 204. Prior to this step, understanding the precise pixel dimensions of your images is crucial to accurately select the required rescaling factor.


In [7]:
#@markdown ##Image rescaling

def tiff_imgs_pairs(base_path, images_folder, factor_xy, output_folder_name):
    ''' rescale .tiff imgs'''


    # paths
    output_folder = os.path.join(base_path, output_folder_name.value)
    if output_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder)

    file_list = os.listdir(images_folder)

    for file_name in file_list:
        if file_name.endswith('.tiff'):
            input_image_path = os.path.join(images_folder, file_name)
            output_image_path = os.path.join(output_folder, file_name)

            # read img
            img = sitk.ReadImage(input_image_path)

            # original dimensions [X, Y, Z]
            original_size = img.GetSize()

            # calculate new dimensions [X, Y]
            new_size = [int(sz * factor_xy) if idx != 2 else sz for idx, sz in enumerate(original_size)]

            # calculate scale factor for new spacing
            scale_factor = [old_size / new_size_val for old_size, new_size_val in zip(original_size, new_size)]

            # calculate new spacing
            new_spacing = [sz * scale_factor_val for sz, scale_factor_val in zip(img.GetSpacing(), scale_factor)]

            # apply resolution rescaling
            resampler = sitk.ResampleImageFilter()
            resampler.SetOutputSpacing(new_spacing)
            resampler.SetSize(new_size)
            resampler.SetOutputDirection(img.GetDirection())
            resampler.SetOutputOrigin(img.GetOrigin())
            resampler.SetTransform(sitk.Transform())
            resampler.SetDefaultPixelValue(img.GetPixelIDValue())

            # execute resampling
            output_image = resampler.Execute(img)

            # save images
            sitk.WriteImage(output_image, output_image_path)

    return output_folder


# rescaling factor
rescaling_factor=0.1#@param{type:"number"}
if rescaling_factor is None:
  print('Please enter a value')
else:
  output_folder =tiff_imgs_pairs(base_path, images_folder, rescaling_factor, output_folder_name=output_folder_name)
  if output_folder:
              print(f'Your rescaled images are in {output_folder}')


Your rescaled images are in /content/gdrive/MyDrive/Colab Notebooks/res/rescaled_imgs
